<a href="https://colab.research.google.com/github/ArasyH/Agentic-Ai-Workshop-Challenge/blob/main/agentic_patterns_challenge_arasy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ARASY HEMASTHIAR
## AGENTIC AI WORKSHOP CHALLENGE

In [ ]:
# !pip install openai-agents
!pip install requests
!pip install groq
!pip install langchain
!pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 2.7 MB/s eta 0:00:00


In [ ]:
import os
import json
import asyncio
import requests

from pydantic import BaseModel
from typing import List, Any
# from agents import Agent, Runner, function_tool, trace
from langchain_groq import ChatGroq

from google.colab import userdata

SECTORS_API_KEY = userdata.get('SECTORS_API_KEY')
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

headers = {"Authorization": SECTORS_API_KEY}

In [ ]:
def retrieve_from_endpoint(url: str) -> dict:

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.HTTPError as err:
        raise SystemExit(err)
    return json.dumps(data)

In [ ]:
from langchain_core.tools import tool


@tool
def get_company_overview(ticker: str, country: str) -> str | None:
    """
    Get company overview from Singapore Exchange (SGX) or Indonesia Exchange (IDX) in specific industries.
    """
    assert country.lower() in ["indonesia", "singapore", "malaysia"], "Country must be either Indonesia, Singapore, or Malaysia"

    if(country.lower() == "indonesia"):
        url = f"https://api.sectors.app/v1/company/report/{ticker}/?sections=overview"
    if(country.lower() == "singapore"):
        url = f"https://api.sectors.app/v1/sgx/company/report/{ticker}/"
    if(country.lower() == "malaysia"):
        url = f"https://api.sectors.app/v1/klse/company/report/{ticker}/"

    try:
        return retrieve_from_endpoint(url)
    except Exception as e:
        print(f"Error occurred: {e}")
        return None

@tool
def get_top_companies_ranked(dimension: str) -> List[str]:
    """
    Return a detail list of top companies (symbol) based on certain dimension (dividend yield, total dividend, revenue, earnings, market_cap, PB ratio, PE ratio, or PS ratio).
    """

    url = f"https://api.sectors.app/v1/companies/top/?classifications={dimension}"

    return retrieve_from_endpoint(url)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
from langchain.agents import create_tool_calling_agent, AgentExecutor

llm = ChatGroq(model_name="llama3-70b-8192", temperature=0, groq_api_key = GROQ_API_KEY)

#Agen 1: Screening perusahaan terbaik berdasarkan dimensi tertentu
screener_tools = [get_top_companies_ranked]
screener_prompt = ChatPromptTemplate.from_messages([
    ("system", """Get the top companies based on the given metric . Return the tickers of the top companies, without the .JK suffix. Return in a list."""),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

get_top_companies_based_on_metric_agent = create_tool_calling_agent(llm, screener_tools, screener_prompt)
get_top_companies_based_on_metric_executor = AgentExecutor(agent=get_top_companies_based_on_metric_agent, tools=screener_tools, verbose=True, return_intermediate_steps=True)



# Agen 2: Researcher agent
research_tools = [get_company_overview]
researcher_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a research assistant specifically for the Indonesian stock market.
- You have one tool, `get_company_overview`.
- The user will provide a company ticker.
- You MUST call the tool with this ticker.
- You MUST set the 'country' parameter to 'Indonesia'.
"""),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])
company_research_agent = create_tool_calling_agent(llm, research_tools, researcher_prompt)
company_research_agent_executor = AgentExecutor(agent=company_research_agent, tools=research_tools, verbose=True, return_intermediate_steps=True, handle_parsing_errors=True)


In [ ]:
async def main():
  """
  Fungsi utama untuk mengorkestrasi alur kerja multi-agent.
  """
  input_prompt = input("🤖: What kind of companies are you interested in? \n💀: ")

  print("\n STEP 1: Screening Companies (Screener Agent) ")
  try:
      response_agent_1 = await get_top_companies_based_on_metric_executor.ainvoke({"input": input_prompt})

      # Fetching raw data from intermediate steps
      intermediate_step_1 = response_agent_1['intermediate_steps'][0]
      tool_output_json_string = intermediate_step_1[1]
      api_data = json.loads(tool_output_json_string)

      dimension_key = list(api_data.keys())[0]
      companies_list = api_data[dimension_key]
      top_companies_tickers = [company['symbol'].replace('.JK', '') for company in companies_list]

      print(f"\n Screener Agent finds tickers: {top_companies_tickers}")

  except (KeyError, IndexError, json.JSONDecodeError, TypeError) as e:
      print(f"\n Failed at Step 1: Unable to parse output from Screener Agent. Error: {e}")
      print(f"Raw output: {response_agent_1}")
      return

  print("\n STEP 2: Detailed Research per Company (Agent 2) ")
  all_results = {}
  for ticker in top_companies_tickers:
      print(f"\n🔎: Getting information on: {ticker}")
      research_input = f"Get an overview for the company with ticker '{ticker}'."

      try:
          company_research_result = await company_research_agent_executor.ainvoke({"input": research_input})

          # # Fetch raw data from the second agent's intermediate_steps
          intermediate_step_2 = company_research_result['intermediate_steps'][0]
          tool_output_json_string_2 = intermediate_step_2[1]
          detailed_info = json.loads(tool_output_json_string_2)
          all_results[ticker] = detailed_info

          print(f"🤖: Info for {ticker} (Data from Sectors API):")
          print(json.dumps(detailed_info, indent=2))

      except (KeyError, IndexError, json.JSONDecodeError, TypeError) as e:
          print(f"\n Failed at Step 2 for ticker {ticker}. Error: {e}")
          print(f"Raw agent output: {company_research_result}")
          continue

  print(f"\n Information for '{input_prompt}' has been presented.")

await main()


🤖: What kind of companies are you interested in? 
💀: Top Companies with best Dividend Yield

 STEP 1: Screening Companies (Screener Agent) 


> Entering new AgentExecutor chain...

Invoking: `get_top_companies_ranked` with `{'dimension': 'dividend_yield'}`


{"dividend_yield": [{"symbol": "DMAS.JK", "company_name": "PT Puradelta Lestari Tbk.", "dividend_yield": 0.202797202797203}, {"symbol": "BJBR.JK", "company_name": "Bank Pembangunan Daerah Jawa Barat dan Banten Tbk", "dividend_yield": 0.100887573964497}, {"symbol": "BNGA.JK", "company_name": "PT Bank CIMB Niaga Tbk", "dividend_yield": 0.0900173385708318}, {"symbol": "NISP.JK", "company_name": "PT Bank OCBC NISP Tbk", "dividend_yield": 0.0776556776556777}, {"symbol": "ASII.JK", "company_name": "Astra International Tbk", "dividend_yield": 0.0668112798264642}]}Here are the top companies with the best dividend yield:

['DMAS', 'BJBR', 'BNGA', 'NISP', 'ASII']

> Finished chain.

 Screener Agent finds tickers: ['DMAS', 'BJBR', 'BNGA', 'NI

if it's failed, then it must be / usually because limit from groq api due to model limits (error 429).
Prompt that i used and successfully run:
1. Top Companies with best Market Cap
2. Top Companies with best Dividend Yield

Success Response Api by ranked companies based on dimensions:
- Market Cap
- Dividend Yield


List that Couldn't provide information dimensions:
- P/B Ratio (error 400)
- Earnings (error 400)
- Revenue (error 400)
- P/E (error 400)
- Ps (error 400)